In [1]:
import sys

import numpy as np

np.random.seed(42)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pandas as pd
import gensim

In [2]:
# In case your sys.path does not contain the base repo, go there.
print(sys.path)
%cd '/Users/axelsirota/repos/ml-solr-course'

['/Users/axelsirota/repos/ml-solr-course/synoyms/lab1', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
path = 'dataset/docv2_train_queries.tsv'
queries = pd.read_csv(path, sep='\t', lineterminator='\r', names=['query_id', 'query'])
queries.head()


/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,query_id,query
0,121352,define extreme
1,510633,tattoo fixers how much does it cost
2,674172,what is a bank transit number
3,570009,what are the four major groups of elements
4,54528,blood clots in urine after menopause


In [4]:
corpus = [sentence for sentence in queries['query'].values if type(sentence) == str and len(sentence.split(' ')) >= 3]

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 3
epochs=50
batch_size = 5000
BATCH = True


In [6]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')


First 5 corpus items are [[1528, 38656, 6, 24, 9, 31, 20], [1, 2, 5, 341, 4411, 43], [1, 11, 3, 913, 329, 2058, 4, 757], [53, 4020, 7, 239, 82, 1948], [1, 2, 1090, 38657]]
Length of corpus is 312265


In [9]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [10]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam')


In [11]:
def generate_data(corpus, window_size, V, batch_size=batch_size):
    number_of_batches = (len(corpus) // batch_size) + 1
    for batch in range(number_of_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = np.zeros((mini_batch_size, maxlen))
        Y = np.zeros((mini_batch_size, V))
        for query_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []            
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = sequence.pad_sequences(contexts, maxlen=maxlen)
                y = np_utils.to_categorical(labels, V)
                X[query_id] = x
                Y[query_id] = y
        yield (X, Y)



In [29]:
# If data is small, you can just generate the whole dataset and load it in memory to use the fit method
#
# def generate_data(corpus, window_size, V):
#         maxlen = window_size*2
#         X = np.zeros((len(corpus), maxlen))
#         Y = np.zeros((len(corpus), V))
#         for query_id, words in enumerate(corpus):
#             L = len(words)
#             for index, word in enumerate(words):
#                 contexts = []
#                 labels   = []            
#                 s = index - window_size
#                 e = index + window_size + 1

#                 contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
#                 labels.append(word)

#                 x = sequence.pad_sequences(contexts, maxlen=maxlen)
#                 y = np_utils.to_categorical(labels, V)
#                 X[query_id] = x
#                 Y[query_id] = y
#         return (X, Y)


In [12]:
def fit_model():
    if not BATCH:
        X, Y = generate_data(corpus, window_size, V)
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        cbow.fit(X, Y, epochs = epochs)
    else:
        index = 1
        for x, y in generate_data(corpus, window_size, V):
            print(f'Training on Iteration: {index}')
            index += 1
            history = cbow.train_on_batch(x, y, reset_metrics=False, return_dict=True)
            print(history)

In [13]:
fit_model()

Training on Iteration: 1


ValueError: in user code:

    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/engine/training.py:771 train_step  *
        loss = self.compiled_loss(
    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/engine/compile_utils.py:201 __call__  *
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/losses.py:142 __call__  *
        losses = call_fn(y_true, y_pred)
    /Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/keras/losses.py:246 call  *
        return ag_fn(y_true, y_pred, **self._fn_kwargs

In [34]:
with open('./lab1/vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(V-1, dim))
    vectors = cbow.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))


In [35]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./lab1/vectors.txt', binary=False)

In [36]:
w2v.most_similar(positive=['gasoline'])

[('for', 0.8791447281837463),
 ('sea', 0.8790562152862549),
 ('solar', 0.878172755241394),
 ('kidney', 0.8773006796836853),
 ('after', 0.8767251372337341),
 ('play', 0.875443696975708),
 ('does', 0.8751270174980164),
 ('savings', 0.875092625617981),
 ('normal', 0.8750559091567993),
 ('temperature', 0.8746627569198608)]

In [38]:
w2v.most_similar(negative=['apple'])

[('thagomizer', 0.4968186616897583),
 ('tomboy', 0.4296087920665741),
 ('stubbing', 0.4010353088378906),
 ('maximization', 0.3943706452846527),
 ('scepter', 0.37567877769470215),
 ('emarket', 0.37318524718284607),
 ('unbuttered', 0.3682001233100891),
 ('romana', 0.35735321044921875),
 ('hubris', 0.3529239594936371),
 ('stouffers', 0.3493630588054657)]